In [ ]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
pandas
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0


!pip install -U --user -r requirements.txt

In [ ]:
USER_SRC_DIR = "src_dir"
!mkdir $USER_SRC_DIR
!mkdir model_artifacts

# copy the requirements to the source dir
!cp requirements.txt $USER_SRC_DIR/requirements.txt

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from google.cloud import storage
from joblib import dump
from sklearn.pipeline import make_pipeline

storage_client = storage.Client()
bucket = storage_client.bucket("vertexai-udemy")

df = pd.read_csv('gs://vertexai-udemy/adclick-model-input/advertising.csv')
max_iter= 1000
solver= 'lbfgs'

X = df[['Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage', 'Male']]
y = df['Clicked on Ad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

logmodel = LogisticRegression(solver=solver,max_iter=max_iter)

pipeline = make_pipeline(logmodel)

pipeline.fit(X_train, y_train)

dump(pipeline, 'model_artifacts/model.joblib')

model_artifact = bucket.blob('adclick-model-artifact/model.joblib')

model_artifact.upload_from_filename('model_artifacts/model.joblib')

In [1]:
# pipeline.predict([['63.95','23','52182','40',1]])

In [2]:
# !gsutil cp model_artifacts/model.joblib {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/

In [ ]:
%%writefile src_dir/predictor.py
import numpy as np
from google.cloud.aiplatform.prediction.sklearn.predictor import SklearnPredictor

class AdclickPredictor(SklearnPredictor):

    def __init__(self):
        return

    def load(self, artifacts_uri: str) -> None:
        super().load(artifacts_uri)

    def postprocess(self, prediction_results: np.ndarray) -> dict:
        return {"predictions": [f"{value}" for value in np.round(prediction_results)]}

In [3]:
REGION = "us-east1"
MODEL_ARTIFACT_DIR = "adclick-model-artifact"
REPOSITORY = "vertexai-repo"
IMAGE = "sklearn-image"
MODEL_DISPLAY_NAME = "adclick-model"

PROJECT_ID = "{your-project-id}"
BUCKET_NAME = "gs://vertexai-udemy"

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

import os

from google.cloud.aiplatform.prediction import LocalModel

from src_dir.predictor import AdclickPredictor

local_model = LocalModel.build_cpr_model(
    "src_dir",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/adclick-image",
    predictor=AdclickPredictor,
    requirements_path=os.path.join("src_dir", "requirements.txt"),
)

In [ ]:
import json

sample = {"instances": [
  ['63.95','23','52182','40',1]
]}

with open('instances.json', 'w') as fp:
    json.dump(sample, fp)


In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri = 'model_artifacts/',
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file='instances.json',
        headers={"Content-Type": "application/json"},
    )
    health_check_response = local_endpoint.run_health_check()

In [ ]:
local_model.push_image()

In [ ]:
model = aiplatform.Model.upload(local_model = local_model,
                                display_name=MODEL_DISPLAY_NAME,
                                artifact_uri=f"{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}",)


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-2",)

In [ ]:
endpoint.predict(instances=[['63.95','23','52182','40',1]])